In [ ]:
import cohortshapleyintegratedgradients as csig
import numpy as np

In [ ]:
#data load
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing()
x = data['data'][:2000]
y = data['target'][:2000]

In [ ]:
#build instance 
IG = csig.CohortIntGrad(x,y)

In [ ]:
#calculate CSIG values and Remaining Delta (residue of efficiency axiom)
ig,rd =IG.igcs_stack(stack_target=list(range(x.shape[0])))

In [ ]:

#calculate original cohort shapley
ks = np.vstack([IG.cohort_kernel_shap(t_id=i) for i in range(x.shape[0])])